In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
 
# Import from Qiskit Aer noise module
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

In [2]:
num_qubits = 4
# gate errors/fidelity
# p_G = [np.random.random(num_qubits)*.05]
p_G = .01

error_global_gate = depolarizing_error(p_G,num_qubits=num_qubits)

# dephasing/relaxation 
T_1 = 50e-5 # relaxation time
# T_2 = 1e-7 # time scale/dephasing
T_2 = 50e-6 # time scale/dephasing
tau_i = 2e-6 # idling time

p_T_1 = tau_i/(4*T_1)
p_T_2 = np.abs((tau_i**2/(2*T_2**2)) - p_T_1)

error_relaxation = pauli_error([("I",1-p_T_1),("X",p_T_1/2),("Y",p_T_1/2)])
error_dephasing = pauli_error([("I",1-p_T_2),("Z",p_T_2)])

# readout 
T_1R = 1e-4 # decoherence time
tau_min = 1.5e-6 # minimum integration time

#readout
p_R = lambda tau_R : 1 - (1-np.exp(-tau_R/tau_min))*np.exp(-tau_R/T_1R)  # noqa: E731

#initialization/reset
# F_init = 0.05
# p_res = (8/15)*(1-F_init)
p_res = .05

tau_R = 1e-6
error_readout_reset = ReadoutError([[p_res*(1-p_R(tau_R)), 1-(p_res*(1-p_R(tau_R)))], [p_R(tau_R)*(1-p_res), 1 - p_R(tau_R)*(1-p_res)]])


In [3]:
spin_noise_model = NoiseModel()
spin_noise_model.add_all_qubit_quantum_error(error_relaxation,"relaxation")
spin_noise_model.add_all_qubit_quantum_error(error_dephasing,"dephasing")
spin_noise_model.add_all_qubit_quantum_error(error_global_gate,"gate")
spin_noise_model.add_all_qubit_readout_error(error_readout_reset,"readout_reset")


In [4]:
sim_noise = AerSimulator(noise_model=spin_noise_model)